# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [4]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_tfidf_feat.columns = X_tfidf_feat.columns.astype(str)
# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)
X_count_feat.columns = X_count_feat.columns.astype(str)
X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
         'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,94.692557,2.969755,0.657112,0.195037,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977558,0.975763,0.976640,0.968553,0.973944,0.974492,0.003200,1
10,61.959126,3.576737,0.677613,0.050281,None,150,"{'max_depth': None, 'n_estimators': 150}",0.978456,0.977558,0.974843,0.967655,0.968553,0.973413,0.004504,2
8,111.884490,4.653576,1.403832,0.536625,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.975763,0.976661,0.973944,0.967655,0.971249,0.973054,0.003273,3
7,47.783681,4.074743,0.829004,0.226449,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.975763,0.973046,0.967655,0.970350,0.973054,0.003820,4
4,36.042975,0.962446,0.593218,0.044530,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.976661,0.972172,0.970350,0.968553,0.972147,0.971977,0.002697,5


In [7]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
         'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,101.761783,1.613710,0.792005,0.194700,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975763,0.974865,0.974843,0.967655,0.973944,0.973414,0.002936,1
10,59.762602,5.057241,0.801448,0.048806,None,150,"{'max_depth': None, 'n_estimators': 150}",0.980251,0.973070,0.973046,0.967655,0.971249,0.973054,0.004104,2
7,57.159675,6.825864,0.906907,0.196243,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.975763,0.972147,0.968553,0.970350,0.973054,0.003603,3
8,113.490288,5.274281,1.209217,0.206689,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.974865,0.973944,0.966757,0.969452,0.972515,0.003887,4
6,6.657904,0.561878,0.535165,0.092220,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.973070,0.978456,0.972147,0.964061,0.969452,0.971437,0.004707,5
